In [1]:
import ee
import geemap

In [2]:
service_account = 'ueinfo@ueinfo.iam.gserviceaccount.com '
credentials = ee.ServiceAccountCredentials(service_account, 'ueinfo-615e315d9158.json')

ee.Initialize(credentials)

In [7]:
india = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq('country_na', 'India'))
geometry = india.geometry()

In [8]:
ind = ee.FeatureCollection("projects/sat-io/open-datasets/VIDA_COMBINED/IND")

In [9]:
buildings = ind.filter(ee.Filter.bounds(geometry));
# Add a dummy property to each feature
buildings = buildings.map(lambda feature: feature.set('dummy', 1))

In [10]:
# // Creating a 1000m grid within the geometry
gridSizeMeters = 10000
 
# // Generate a rectangular grid
#grid = geometry.coveringGrid('EPSG:3857', gridSizeMeters)

grid = ee.Image().toInt().paint(india, 1).reproject(crs='EPSG:4326', scale=gridSizeMeters)

In [11]:
# Rasterize the building footprints to create a density map
building_raster = buildings.reduceToImage(properties=["dummy"], reducer=ee.Reducer.count()).unmask(0)

# Clip the raster to the India boundary
building_raster = building_raster.clip(india)

# Convert the building raster to a double type
building_raster = building_raster.toDouble()

In [12]:
# Set the default projection for the building raster
building_raster = building_raster.setDefaultProjection(crs='EPSG:4326', scale=gridSizeMeters)

In [13]:
# Aggregate the building count to 1000m grid cells
building_density = building_raster.reduceResolution(
    reducer=ee.Reducer.sum(),
    bestEffort=True).reproject(crs='EPSG:4326', scale=gridSizeMeters)

In [14]:
geemap.ee_export_image(building_density,
                       filename='IndiaBD10000.tif',
                        scale=gridSizeMeters,
                        region=geometry,
                        file_per_band=True)

Generating URL ...
Please wait ...
An error occurred while downloading.


In [15]:
# Define the export task
export_task = ee.batch.Export.image.toDrive(
    image=building_density,
    description='BuildingDensityIndia',
    folder='EarthEngineExports',  # Google Drive folder name
    fileNamePrefix='building_density_india',
    scale=gridSizeMeters,
    region=india.geometry().bounds(),
    crs='EPSG:4326')

# Start the export task
export_task.start()

In [16]:
# Monitor the task
import time
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)

print('Export task completed.')

Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id: 6Q7NSPDF4EIDTPL7SGWLNJXH).
Polling for task (id

ConnectionError: HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Max retries exceeded with url: /v1/projects/earthengine-legacy/operations/6Q7NSPDF4EIDTPL7SGWLNJXH?alt=json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000216FD8FB1D0>: Failed to resolve 'earthengine.googleapis.com' ([Errno 11001] getaddrinfo failed)"))